<img src='https://github.com/ec-jrc/cbm/blob/main/docs/img/eu_science_hub.png?raw=true' alt="EC JRC" width="1000">


# JRC RESTful service example

This Jupyter Notebook provides examples on how to retrieve and view data from the JRC RESTful API.


This file is part of CbM (https://github.com/ec-jrc/cbm). All code examples are under the open source License [3-Clause BSD](https://opensource.org/licenses/BSD-3-Clause)

For more information See the documentation at: https://jrc-cbm.readthedocs.io


---
## Get data with python requests

Use the python library [```requests```](https://pypi.org/project/requests/) to manually configure and make requests.
The https://cap.users.creodias.eu/query/info requests provides information and requests examples in json format.

In [ ]:
# RESTful API Requests example with the use of requests library
# The '/query/info' requests will return information about the server and the available data.
import json
import requests

api_url  = 'https://cap.users.creodias.eu'
api_user = 'USERNAME'
api_pass = 'PASSWORD'

requrl = "{}/query/info"
response = requests.get(requrl.format(api_url),
                        auth=(api_user, api_pass))

print(json.dumps(json.loads(response.content), indent=1))

---

## Get and view data with the python tool 'cbm' 
Python ```cbm``` is a tool to easily explore the data and check individual parcels or small set of parcels.

See the documentation for more details at: https://jrc-cbm.readthedocs.io/en/latest/cbm_overview.html

In [ ]:
# Install or upgrade cbm (Restart of the Notebook kerner may be required to take effect)
!pip install cbm --upgrade

In [ ]:
import cbm
print("Python cbm version: ", cbm.__version__)

In [ ]:
cbm.set_api_account(api_url, api_user, api_pass) # set and store your credentials.

---
## Show data for a selected parcel
- Provide general information for the selected parcel

In [ ]:
# Select parcel
import cbm
aoi   = 'MS'        # Area of interest (str)
year  = '2020'      # Year of the parcels dataset (int)
pid   = '123456'    # Parcel id (int)
ptype = ''          # Parcel type (Optional), use only in case there are separate parcel datasets for
                    #   the same year dedicated to different type of analyses or different regions),
                    #   to see the available parcel types go to: https://cap.users.creodias.eu/query/info

cbm.show.parcel_info.by_pid(aoi, year, pid, ptype)

- Show parcel backround images. For some member states or regions orthophotos are provided as well.

In [ ]:
# To see the available orthophotos go to: https://cap.users.creodias.eu/query/info
cbm.show.background.by_pid(
    aoi=aoi,
    year=year,
    pid=pid,
    chipsize=256,  # Pixels
    extend=500.0,  # Meters
    tms=['google', 'bing', 'ags', 'osm'],
    ptype=ptype
)


### Sentinel 2 images
- Show Sentinel images in a grid: true color.

In [ ]:
# This takes some time to download the images, do not use this for large amount of parcels or long time periods.
chipsize = 512
start_date = '2020-04-05'
end_date = '2020-04-25'
imgtype = 'true color' # imgtype options:
#                            - Multi band, e.g.:['B03', 'B04', 'B08']
#                            - Single band, e.g: 'B08'
#                            - Or 'true color' and 'ndvi'

# # Other default options:
# clean_history = False # Delete the previous chip_images folder
# debug = False         # Show text output of the procedures
# columns = 4           # Number of columns of the grid
# show_parcel = True    # Show or not the parcel polygon
# percentile = [2, 98]  # Color levels percentage
# cmap = 'RdYlGn_r'     # Color map only for NDVI

cbm.show.chip_images.grid(aoi, year, pid, start_date, end_date, imgtype, chipsize, ptype=ptype)

- Show Sentinel images in a grid: NDVI

In [ ]:
# Show NDVI images in a grid.
imgtype = 'ndvi'
cbm.show.chip_images.grid(aoi, year, pid, start_date, end_date, imgtype, chipsize, ptype=ptype)

### Sentinel 1/2 signatures time series pllots
- Sentinel 2 band profiles for the selected parcel.

In [ ]:
cbm.show.time_series.s2(aoi, year, pid, bands=['B02', 'B03', 'B04', 'B08'], scl='3_8_9_10_11', ptype=ptype)

- NDVI profile.

In [ ]:
cbm.show.time_series.ndvi(aoi, year, pid, ptype=ptype)

- Sentinel-1 Backscattering coefficient.

In [ ]:
cbm.show.time_series.s1(aoi, year, pid, 'bs', ptype)

- Sentinel-1 6 day coherence.

In [ ]:
cbm.show.time_series.s1(aoi, year, pid, 'c6', ptype)

---

### Compare neighbour parcels with same crop type

- List of parcels with the same crop type within a given distance.

In [ ]:
# Find parcels with the same crop type within a given distance.
parcel_peers = cbm.get.parcels_list.peers(aoi, year, pid, distance=1000, maxPeers=5, ptype=ptype)
parcel_peers

- NDVI profile plots of the parcels peers.

In [ ]:
cbm.show.time_series.ndvi(aoi, year, parcel_peers['pids'], ptype=ptype)

- Compare the profiles of the selected parcel with its parcel peers.

In [ ]:
cbm.show.time_series.ndvi(aoi, year, [pid]+parcel_peers['pids'], ptype=ptype, max_line_plots=1)

- Sentinel-1 Backscattering coefficient plots with its parcel peers.

In [ ]:
cbm.show.time_series.s1(aoi, year, [pid]+parcel_peers['pids'], 'bs', ptype, max_line_plots=1)

---

## Exporting results

There are two methods to generate reports
- __from_notebook()__: convert a notbook to pdf (easy but with limited customization)
- __with_pdfme()__: generate pdf with the pdfme module (complex but more customizable)

### From notebook

The easiest method to create reports is with the __cbm module__ ```from_notebook```. It will generate a pdf report with all the outputs of the cells, provides the option to include the code cells as well.

__Note:__ This function generates a pdf document from the saved file therefore the __Notebook must be saved__ before running the function to get the latest changes.
```python
import cbm
cbm.reports.export_to_pdf.from_notebook()
```

Parameters:
- with_code: show code cels or not ```True``` or ```False```, (default ```False```)
- nb_fname: Notebook name, the notebook to be converted, if ```None``` it will check if the script is running in a notebook and will try to get its name (default ```None```)

    ```e.g.: cbm.reports.export_to_pdf.from_notebook(True, 'MyNotebook.ipynb')```


#### With pdfme

```python
notes = {"general": "My notes", ...}
params = {"textsize": 12, ...}
cbm.reports.export_to_pdf.with_pdfme(aoi, year, pid, ptype, notes, params)
```

In [ ]:
# Exporting results from notebook
import cbm
cbm.reports.export_to_pdf.from_notebook()

---
# Jupyter notebooks interactive widgets for CbM **'ipycbm'**

The ipycbm subpackage of cbm python library provides an easy way to get and view data with the use of python widgets for notebooks 'ipywidgets' also known as jupyter-widgets, the interactive HTML widgets for Jupyter notebooks and the IPython kernel.
For more details see see the ["CbB overview"](https://jrc-cbm.readthedocs.io/en/latest/cbm_overview.html) chapter of the documentation.

__Note:__ the ```ipycbm``` module is interactive therefore not suitable for generating reports from notebooks (_from_notebook()_), ipycbm will provide options to generate customizable reports within interactive widgets (under development), please follow the developments at https://github.com/ec-jrc/cbm.

To see the interactive widget run the main ipycbm widget with:
```python
from cbm import ipycbm
ipycbm.view()
```

In [ ]:
from cbm import ipycbm # Import the python library ipycbm
ipycbm.view()          # Display the data with easy selectable predefined view options.

---

    Copyright : 2022 European Commission, Joint Research Centre